In [ ]:
import os
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from env import get_db_url

In [ ]:
#Write some SQL - expect 2152863 rows
sql = """
SELECT bedroomcnt as bed,
    bathroomcnt as bath, 
    calculatedfinishedsquarefeet as sf, 
    taxvaluedollarcnt as value, 
    yearbuilt, 
    taxamount, 
    fips
FROM properties_2017
    JOIN propertylandusetype USING(propertylandusetypeid)
WHERE propertylandusedesc = 'Single Family Residential';
"""


In [ ]:
df = df_backup = pd.read_sql(sql,get_db_url('zillow'))

In [ ]:
df.shape #rows match

In [ ]:
df.info(show_counts=True)

### Investigating the nulls

First, let's look at the nulls of our target variable

In [ ]:
df[df.value.isna()].shape

Only 500 of 2 million, plus we don't want to be imputing our target. 

**ACTION:** Drop all rows where target variable is null

In [ ]:
value_null_ind =  df[df.value.isna()].index
df.drop(index=value_null_ind,inplace=True)
df.info(show_counts=True)

Based on the above table, we have fips for every row which makes sense as every parcelid (pk from orginial table) should have a fips.  

The next highest non-null columns are bed and bath.  so let's investigate those first

In [ ]:
df[df.bed.isna()]

Looking at the above table, this subset of rows have minimal information.  If there is no bed or bath AND half of the remaining columns don't exist, we should just drop.

**ACTION:** Drop all rows where df.bed is null

In [ ]:
bed_null_ind = df[df.bed.isna()].index
df.drop(index=bed_null_ind,inplace=True)
df.info(show_counts=True)

Now let's see how many are in the rest of the columns

In [ ]:
df[df.sf.isna()].shape

In [ ]:
df[df.yearbuilt.isna()].shape

In [ ]:
df[df.taxamount.isna()].shape

In [ ]:
#let's check out some overlap
hist = df.count(axis=1,numeric_only=False)
plt.hist(hist[hist<7])

Looking at the above, a decent number of these are only missing one piece of data.  However, since we are still only looking at ~12k total rows, I am going to go ahead and drop them all.  Ideally, I'd check some of this with stakeholders.  My primary concern is that we may be inadvertently trimming a particular geographic area (perhaps one with poor tax amount reporting to the county)

**ACTION:** Drop all nulls

In [ ]:
df.dropna(inplace=True)
df.shape

### Look at the distributions of each feature

In [ ]:
#See on logarythmic scale to better see outliers
plt.figure(figsize=(10,15))
ct=0
for c in df.columns:
    ct +=1
    plt.subplot(7,2,ct)
    plt.hist(df[c])
    plt.title(c+'_log')
    plt.yscale('log')
    ct +=1
    plt.subplot(7,2,ct)
    plt.hist(df[c])
    plt.title(c)
    plt.ylim((0,10))
    
plt.tight_layout()

Definitely some bed, bath, tax, value and year outliers.  I don't want to snap any values in.  I also think it may be best to ignore bed/bath outliers first, then see if addressing sf or value will handle those.

I want to avoid trimming by my target variable, so first I'll trim by sf and hope that it also helps address outliers on the value:
- Since Zillow estimates are not expected to be utilized by the top 1%, I want to cut some of the expensive houses. We'll cut the top .1% of sf homes off the dataset.  NOTE: I recognized that sf is being used as a proxy for value, but it's one method of avoiding trimming by our target so that we can better specify how the model was trained in a useful manner
- Since the data is skewed right, I want to trim less off the left.  After considering a few different cutoffs, I found that california code restricts minimum dwelling size to 120 sq ft. 
  - "Every dwelling unit shall have at least one room that shall have not less than 120 square feet (13.9 m2) of net floor area"
  
**ACTION:** Drop rows with the top .1% of sf or an sf of less than 120

In [ ]:
#drop top 1% of sf
df = df[df.sf<df.sf.quantile(.999)]

#drop anything less than 120 sf
df = df[df.sf>=120]
df.shape

In [ ]:
df.describe().T

In [ ]:
#See on logarythmic scale to better see outliers
plt.figure(figsize=(10,15))
ct=0
for c in df.columns:
    ct +=1
    plt.subplot(7,2,ct)
    plt.hist(df[c])
    plt.title(c+'_log')
    plt.yscale('log')
    ct +=1
    plt.subplot(7,2,ct)
    plt.hist(df[c])
    plt.title(c)
    plt.ylim((0,10))
    
plt.tight_layout()

Even after using sf trimming, we still have quite a few outliers in each category.  Because of that, I'll do more trimming on the high side.  Since Zillow's target customers aren't the super rich, I feel comfortable trimming 10+ bedrooms, 10+ bathrooms and 2+ million value.  While we wanted to avoid any triming by value, it is import to get rid of these extreme outliers.


In [ ]:
df[df.bed > 9].shape

In [ ]:
df[df.bath>9].shape

In [ ]:
df[df.value>2_000_000].shape

In [ ]:
df[(df.value>2_000_000) | (df.bath >9) | (df.bed >10)].shape[0] / df.shape[0]

Even with the extra trimming, that only accounts for <2% of the data.  

**Action:** Drop all rows with 10+ beds, 10+ baths, or a value of 10+ million.

In [ ]:
df = df[(df.value < 2_000_000) & (df.bath < 10) & (df.bed <10)]
df.shape

### Now encode our categorical

In [ ]:
df.fips.value_counts()

In [ ]:
#only categorical is FIPS.  I want to map then encode so that I have common sense 
# names for EDA and easy to read columns for the model

#map to county names
df['county'] = df.fips.map({6037: 'LosAngeles_CA',6059:'Orange_CA',6111:'Ventura_CA'})
#encode into dummy df
d_df = pd.get_dummies(df['county'],drop_first=True)
#concat dummy df to the rest
df = pd.concat([df,d_df],axis=1)

In [ ]:
df.info(show_counts=True)

### See if other datatypes are appropriate


In [ ]:
df.bed.value_counts() #can be integer

In [ ]:
(df.yearbuilt % 1).value_counts() # can be integer

Convert those two to integers

In [ ]:
df.bed = df.bed.astype(int)
df.yearbuilt = df.yearbuilt.astype(int)
df.info()

### Drop any unecessary columns

taxamount is proportional to tax value.  In addition, tax amount follows tax value.  Major changes in tax amount often come after the sale of a home.  So for Zillow's purposes, we don't want this as a predictor.

In [ ]:
#Get rid of original fips column
df.drop(columns=['fips','taxamount'],inplace=True)

In [ ]:
df.info()

### Now reorder columns for easier EDA/model splits

In [ ]:
df.columns

In [ ]:
df = df.reindex(columns=['value', 'county', 'bed', 'bath', 'sf', 'yearbuilt', 'Orange_CA', 'Ventura_CA'])

In [ ]:
df.info()

### Dropped work into function in wrangle.py

### Now test the functions

In [ ]:
import wrangle

In [ ]:
test_df = wrangle.getZillowData()

In [ ]:
test_df.info(show_counts=True)

In [ ]:
tr, te, val = wrangle.prep_zillow(test_df)

In [ ]:
tr.info()